<a href="https://colab.research.google.com/github/Ferrariagustinpablo/Data-Analytics-in-R/blob/main/Case_Study_Joins_on_Stack_Overflow_Dataset_DataCamp_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [47]:
# Load
library(dplyr)
library(ggplot2)
library(tidyr)
library(ggplot2)
library(forcats) # for use of fct_reorder

# Importings

In [48]:
# Reading datasets
questions = readRDS("/content/questions.rds")
str(questions)

question_tags = readRDS("/content/question_tags.rds")
str(question_tags)

tags = readRDS("/content/tags.rds")
str(tags)

answers = readRDS("/content/answers.rds")
str(answers)

spec_tbl_df [294,735 × 3] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ id           : int [1:294735] 22557677 22557707 22558084 22558395 22558613 22558677 22558887 22559180 22559312 22559322 ...
 $ creation_date: Date[1:294735], format: "2014-03-21" "2014-03-21" ...
 $ score        : int [1:294735] 1 2 2 2 0 2 8 1 0 2 ...
tibble [497,153 × 2] (S3: tbl_df/tbl/data.frame)
 $ question_id: int [1:497153] 22557677 22557677 22557677 22557677 22558084 22558084 22558395 22558395 22558395 22558395 ...
 $ tag_id     : int [1:497153] 18 139 16088 1672 6419 92764 5569 134 9412 18621 ...
spec_tbl_df [48,299 × 2] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ id      : num [1:48299] 124399 124402 124404 124405 124407 ...
 $ tag_name: chr [1:48299] "laravel-dusk" "spring-cloud-vault-config" "spring-vault" "apache-bahir" ...
 - attr(*, "spec")=List of 3
  ..$ cols   :List of 2
  .. ..$ Id     : list()
  .. .. ..- attr(*, "class")= chr [1:2] "collector_double" "collector"
  .. ..$ TagName: list()
  .. .. ..-

# Create a merged dataframe

In [49]:
# Join the questions and question_tags tables
merged <- questions %>%
  left_join(question_tags, by = c("id" = "question_id")) %>%
  left_join(tags, by = c("tag_id" = "id")) %>%
  replace_na(list(tag_name = "only-r", tag_id= 0))

# Comparing scores across tags

In [50]:
merged %>%
	# Group by tag_name
	group_by(tag_name) %>%
	# Get mean score and num_questions
	summarize(score = mean(score),
          	  num_questions = n()) %>%
	# Sort num_questions in descending order
	arrange(desc(num_questions)) %>%
  head(4)

tag_name,score,num_questions
<chr>,<dbl>,<int>
only-r,1.257926,48541
ggplot2,2.613433,28228
dataframe,2.314348,18874
shiny,1.453689,14219


# What tags never appear on R questions? Use anti_join

In [51]:
# Using a join, filter for tags that are never on an R question
tags %>%
  anti_join(question_tags, by = c("id" = "tag_id")) %>%
  dim()

[1] 40459     2

# Finding gaps between questions and answers

In [52]:
questions %>%
	# Inner join questions and answers with proper suffixes
	inner_join(answers, by = c("id" = "question_id"), suffix = c("_question", "_answer")) %>%
	# Subtract creation_date_question from creation_date_answer to create gap
	mutate(date_gap = as.integer(creation_date_answer - creation_date_question)) %>%
  head(2)

id,creation_date_question,score_question,id_answer,creation_date_answer,score_answer,date_gap
<int>,<date>,<int>,<int>,<date>,<int>,<int>
22557677,2014-03-21,1,22560670,2014-03-21,2,0
22557707,2014-03-21,2,22558516,2014-03-21,1,0


# Joining question and answer counts

In [54]:
# Count and sort the question id column in the answers table
answer_counts <- answers %>%
	count(question_id, sort = TRUE)

In [55]:
# Combine the answer_counts and questions tables
questions %>%
	left_join(answer_counts, by = c("id" = "question_id")) %>%
	# Replace the NAs in the n column
	replace_na(list(n = 0)) %>%
  head(2)

id,creation_date,score,n
<int>,<date>,<int>,<dbl>
22557677,2014-03-21,1,1
22557707,2014-03-21,2,2


# Binding rows (like append in python)

In [62]:
dim(questions)
dim(answers)

# Binding rows.
questions %>%
  bind_rows(answers) %>%
  dim()

[1] 294735      3

[1] 380643      4

[1] 675378      4